In [1]:
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = "3"
device = torch.device("cuda")

In [2]:
import sys
sys.path.append('./huggingface_models/')

In [3]:
from run import *
from compilation_test_utils import *
from tokenization_utils import *
from bleu import *

ModuleNotFoundError: No module named 'utils.compilation_test_utils'

In [ ]:
# Original notebook can be found in 
# dhruv /home/aneesh/huggingface_models

### Select Data

In [5]:
program_data = home_path + "g4g/XLCoST_data/pair_data_tok_full/"
small_test_data = "./small_test_data/"
program_data = small_test_data
lang1 = "Java"
lang2 = "Python"

### Select Model

In [7]:
# model_type = "codet5"
# model_name_or_path = "Salesforce/codet5-base"

model_type = "plbart"
model_name_or_path = "uclanlp/plbart-python-en_XX" #"uclanlp/plbart-base" #uclanlp/plbart-python-en_XX
load_model_path_prefix = model_type + "_program/" + lang1 + "-" + lang2 + '/'
load_model_path = load_model_path_prefix + "checkpoint-best-bleu/pytorch_model.bin"
config_class, model_class, tokenizer_class = MODEL_CLASSES[model_type]
config = config_class.from_pretrained(model_name_or_path)
tokenizer = tokenizer_class.from_pretrained(model_name_or_path)
model = model_class.from_pretrained(model_name_or_path)
model.load_state_dict(torch.load(load_model_path))
model.eval()
model.to(device)
print()

<All keys matched successfully>

In [9]:
lang_pair = lang1 + "-" + lang2
if not os.path.isdir(program_data + lang_pair):
    lang_pair = lang2 + "-" + lang1
test_file_prefix = program_data + lang_pair + "/test-" + lang_pair + "-tok"
test_src_file = test_file_prefix + file_extensions[lang1]
test_tgt_file = test_file_prefix + file_extensions[lang2]
test_file = test_src_file + "," + test_tgt_file
output_dir = './'


sys.argv = ['pl', '--do_test', 
    '--model_type', model_type, 
    '--model_name_or_path', model_name_or_path, 
    '--config_name', model_name_or_path, 
    '--tokenizer_name', model_name_or_path, 
    '--test_filename', test_file,
    '--output_dir', output_dir, 
    '--max_source_length', '400', 
    '--max_target_length', '400', 
    '--beam_size', '5', 
    '--eval_batch_size', '64']

In [3]:
parser = get_args_parser()
args = parser.parse_args()

NameError: name 'argparse' is not defined

In [ ]:
from bleu import *
beam_size, do_sample, temperature = 5, False, 0.5
eval_examples, eval_dataloader = get_eval_dataloader(test_file, args.eval_batch_size)
out_fn = "output." 
if do_sample:
    out_fn += str(int(temperature*100)) + ".sample"
else:
    out_fn += str(beam_size) + ".beam"
sample_generation(eval_examples, eval_dataloader, model, args, beam_size, do_sample, temperature, out_fn, 5)

In [18]:
# Pipeline:
# Sample
# Detok
# compile
# test
# generate table results

### Beam Search, Tempering, Sampling

In [ ]:
eval_batch_size = 32
# bs_list = [32, 16, 8, 4, 4]
beam_list = [1, 5, 10, 15, 20]
temper_list = [0.25, 0.5, 0.75, 0.95]
sample_size_list = [1, 5, 10, 15, 20]
for bs in beam_list:
    beam_size, do_sample, temperature = bs, False, 0.25
    print(beam_size, do_sample, temperature)
    compile_samples(eval_batch_size, beam_size, do_sample, temperature)
for temper in temper_list:
    beam_size, do_sample, temperature = 1, True, temper
    print(beam_size, do_sample, temperature)
    compile_samples(eval_batch_size, beam_size, do_sample, temperature)
    
sample_size_list = [5, 10, 15, 20]
for sample_size in sample_size_list:
    beam_size, do_sample, temperature = 1, True, 0.5
    print(beam_size, do_sample, temperature, sample_size)
    compile_samples(eval_batch_size, beam_size, do_sample, temperature, sample_size)

In [104]:
torch.cuda.empty_cache()

### Test Model Inference

In [9]:
model.eval()
model.to(device)

eval_batch_size = 16
max_source_length, max_target_length = 400, 400
test_file, eval_examples, eval_dataloader = get_eval_dataloader(program_data, lang1, lang2, eval_batch_size, 
                                                     max_source_length, max_target_length, tokenizer)
out_fn = "output.0.beam" 
sample_generation_single(test_file,eval_examples, eval_dataloader, model, model_type, tokenizer, 
                         max_target_length, out_fn, device)


100%|██████████| 57/57 [05:24<00:00,  5.70s/it]


  bleu-4 = 35.72 
  xMatch = 0.3333 
  ********************


### New Baselines

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_type = "incoder"
model_name_or_path = "facebook/incoder-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.add_special_tokens({'pad_token': '[PAD]', 
                              'cls_token':tokenizer.bos_token, 'sep_token':tokenizer.bos_token})
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model.train()
model.to(device)
print()

tokenizer("haha, lol", max_length=2, padding="max_length", truncation=True)
eval_examples, eval_dataloader = get_eval_dataloader(test_file, 4)
for batch in eval_dataloader:
# batch = (eval_dataloader)
    batch = tuple(t.to(device) for t in batch)
    source_ids,source_mask = batch
    outputs = model(*[source_ids,source_mask,source_ids,source_mask]) #, decoder_attention_mask=source_mask
    loss = outputs.loss
    break